In [1]:
import os
from pyTigerGraph import TigerGraphConnection
from dotenv import load_dotenv

load_dotenv()
# We first create a connection to the database
host = os.environ["HOST"]
username = os.getenv("USERNAME", "tigergraph")
password = os.getenv("PASS", "tigergraph")
conn = TigerGraphConnection(
    host=host,
    username=username,
    password=password,
)
# conn.getToken()

# And then add CoPilot's address to the connection. This address
# is the host's address where the CoPilot container is running.
# conn.ai.configureCoPilotHost("http://localhost:8000")

In [ ]:
conn.gsql("""CREATE GRAPH SupportAIDocIngestion()""")

In [2]:
conn.graphname = "SupportAIDocIngestion"
conn.getToken()

'l11vnumq77c33f0aa2ss5m0th5hqdj14'

In [3]:
conn.ai.configureCoPilotHost("http://localhost:8000")
# Create Graph Schema
# Install GSQL queries
# conn.ai.initializeSupportAI()

In [ ]:
access = os.environ["AWS_ACCESS_KEY_ID"]
sec = os.environ["AWS_SECRET_ACCESS_KEY"]
res = conn.ai.createDocumentIngest(
    data_source="s3",
    data_source_config={"aws_access_key": access, "aws_secret_key": sec},
    loader_config={"doc_id_field": "url", "content_field": "content"},
    file_format="json",
)

In [ ]:
conn.ai.runDocumentIngest(res["load_job_id"], res["data_source_id"], "s3://tg-documentation/pytg_current/pytg_current.jsonl")

In [4]:
conn.ai.forceConsistencyUpdate(method="supportai")

{'status': 'submitted'}

In [ ]:
query = "How do I get a count of vertices in Python?"

### HNSW Index Overlap in Graph

In [ ]:
conn.ai.searchDocuments(query,
                        method="hnswoverlap",
                        method_parameters = {"indices": ["Document", "DocumentChunk", "Entity", "Relationship"],
                                             "top_k": 2,
                                             "num_hops": 2,
                                             "num_seen_min": 2})

### Document Chunk Vector Search

In [ ]:
conn.ai.searchDocuments(query,
                        method="vdb",
                        method_parameters={"index": "DocumentChunk",
                                           "top_k": 5,
                                           "withHyDE": False})

### Sibling Document Chunk Vector Search

In [ ]:
conn.ai.searchDocuments(query,
                        method="sibling",
                        method_parameters={"index": "DocumentChunk",
                                           "top_k": 5,
                                           "lookahead": 3,
                                           "lookback": 3,
                                           "withHyDE": False})

## Comparing LLM Generated Responses

TigerGraph CoPilot provides a way to generate the response to the user's query using a LLM, based on the search results from the methods above. You can compare the responses generated by the LLM for each of the search methods to see which one is the most relevant to the user's query. In this example, we can see that the HNSW Overlap method generates the most relevant response to the user's query. While none of the responses were wrong, the HNSW Overlap method generated the most relevant response to the user's query, by suggesting to use the `getVertexCount()` function to get the number of vertices in the graph.

In [ ]:
resp = conn.ai.answerQuestion(query,
                        method="hnswoverlap",
                        method_parameters = {"indices": ["Document", "DocumentChunk", "Entity", "Relationship"],
                                             "top_k": 2,
                                             "num_hops": 2,
                                             "num_seen_min": 2})

In [ ]:
print(resp["response"])

In [ ]:
print(resp["retrieved"])

In [ ]:
resp = conn.ai.answerQuestion(query,
                        method="vdb",
                        method_parameters={"index": "DocumentChunk",
                                           "top_k": 5,
                                           "withHyDE": False})

In [ ]:
print(resp["response"])

In [ ]:
print(resp["retrieved"])

In [ ]:
resp = conn.ai.answerQuestion(query,
                        method="sibling",
                        method_parameters={"index": "DocumentChunk",
                                           "top_k": 5,
                                           "lookahead": 3,
                                           "lookback": 3,
                                           "withHyDE": False})

In [ ]:
print(resp["response"])

In [ ]:
print(resp["retrieved"])